# About this notebook

Image blending:
- Overlaying
- Linear blending
- Overlaying + gaussian filter
- Linear blending + cv inpaint
- Overlaying + HuggingFace inpainting
- Overlaying + Dall-e 2 inpainting

# Environment

In [ ]:
import os

import cv2 as cv
import matplotlib.pylab as plt
import numpy as np
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline

# Functions

In [ ]:
def show_result(
    patch: np.array, 
    background: np.array, 
    output: np.array,
    zoom: tuple[tuple[int, int]],
    method_name: str
):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    axes[0].imshow(patch)
    axes[0].set_title("Patch")
    axes[1].imshow(background)
    axes[1].set_title("Background")
    axes[2].imshow(output)
    axes[2].set_title("Merged images")
    axes[3].imshow(output[
        zoom[1][0]:zoom[1][1],
        zoom[0][0]:zoom[0][1],
        :
    ])
    axes[3].set_title("Zoomed")
    # axes[3].set_xlim(zoom[0])
    # axes[3].set_ylim(zoom[1])
    plt.suptitle(method_name)

# Parameters

In [ ]:
DATA_DIR = "../../data/postprocessed"

BORDER = 20 # in pixel

# Load example

In [ ]:
patch_path = os.path.join(
    DATA_DIR,
    "train",
    "patches",
    "20220816_TaenikonWiese_S_xx_F_xx_O_sama_ID2_DJI_20220816121514_0132.0_2_rumex.png"
)
easy_background_path = os.path.join(
    DATA_DIR,
    "train",
    "images",
    "20230615_SchildDotnachtZaelgli_S_20_F_60_H_12_O_krma_ID1_DJI_20230615145252_0193.1_3.png"
)
difficult_background_path = os.path.join(
    DATA_DIR,
    "train",
    "images",
    "20230609_HerrenpuentSuedost_S_20_F_60_H_12_O_krma_ID1_DJI_20230609151113_0028.1_3.png"
)

In [ ]:
suffix = "difficult"

In [ ]:
background_path = easy_background_path

In [ ]:
patch = cv.imread(patch_path)
patch = cv.cvtColor(patch, cv.COLOR_RGB2BGR)
background = cv.imread(background_path)
background = cv.cvtColor(background, cv.COLOR_RGB2BGR)

In [ ]:
# Background must be a square for some of the methods we are trying
background = background[:512, :512, :]

In [ ]:
# Make sure the patch is small in comparison to the background image
bg_y, bg_x, _ = background.shape
# scaling = 0.2
# patch = cv.resize(patch, (int(bg_x * scaling), int(bg_y * scaling)))
p_x, p_y, _ = patch.shape

In [ ]:
# Select an area in the background where we want to add the patch
x, y = 50, 50
x_min, x_max = x, x+p_x
y_min, y_max = y, y+p_y
bg_patch = background[x_min:x_max, y_min:y_max, :]

In [ ]:
assert bg_patch.shape == patch.shape

In [ ]:
ZOOM = ((y_min-BORDER, y_max+BORDER), (x_min-BORDER, x_max+BORDER))

In [ ]:
bounding_box_mask = np.zeros(background.shape[:-1])
bounding_box_mask[x_min:x_max, y_min:y_max] = 1

# Overlaying patch

In [ ]:
# Add blended images pack into the background
background_overlay = background.copy()
background_overlay[x_min:x_max, y_min:y_max, :] = patch

In [ ]:
method_name = "Simple Overlaying"
show_result(
    patch, 
    background,
    background_overlay, 
    zoom=ZOOM,
    method_name=method_name
)
plt.savefig(f"{method_name.replace(' ', '_')}_{suffix}.png", dpi=400)

# Linear blending

In [ ]:
# Blend patch and subset of background
alpha = 0.7
beta = 1 - alpha
dst = cv.addWeighted(patch, alpha, bg_patch, beta, 0)

In [ ]:
# Add blended images pack into the background
background_linear_blend = background.copy()
background_linear_blend[x:x+p_x, y:y+p_y, :] = dst

In [ ]:
method_name="Linear Blending"
show_result(
    patch, 
    background, 
    background_linear_blend, 
    zoom=ZOOM,
    method_name=method_name
)
plt.savefig(f"{method_name.replace(' ', '_')}_{suffix}.png", dpi=400)

# Overlay + Gaussian blur

- Blurring edges of overlay

In [ ]:
# Mask is essentially the border around the patch
border = int(0.2 * min(p_x, p_y))  # in pixel
mask = np.zeros((p_x+2*border, p_y+2*border))
mask[:2*border, :] = 1
mask[-2*border:, :] = 1
mask[:, -2*border:] = 1
mask[:, :2*border] = 1

tmp = np.zeros((bg_y, bg_x))
tmp[x-border:x+p_x+border, y-border:y+p_y+border] = mask
mask = tmp

In [ ]:
# Convert mask to 3D mask
mask_3d = np.dstack((mask, mask, mask))

In [ ]:
mask_image = (mask_3d * 255).astype(np.uint8)

In [ ]:
border_smoothed = cv.GaussianBlur(
    background_overlay,
    (0, 0),
    sigmaX=1,
    sigmaY=1,
    borderType=cv.BORDER_DEFAULT
)

In [ ]:
# Add blended images pack into the background
background_overlay_blurred = background_overlay.copy()
for i in range(background_overlay_blurred.shape[0]):
    for j in range(background_overlay_blurred.shape[1]):
        if mask[i, j]:
            background_overlay_blurred[i, j] = border_smoothed[i, j]

In [ ]:
method_name="Overlaying and Gaussian Smoothing"
show_result(
    patch, 
    background, 
    background_overlay_blurred, 
    zoom=ZOOM,
    method_name=method_name
)
plt.savefig(f"{method_name.replace(' ', '_')}_{suffix}.png", dpi=400)

# Linear blending + Inpainting

In [ ]:
background_inpaint = cv.inpaint(background_overlay, mask.astype(np.uint8), 3, cv.INPAINT_NS) # cv.INPAINT_TELEA)

In [ ]:
method_name="Overlay + Simple Inpaint"
show_result(
    patch, 
    background, 
    background_inpaint, 
    zoom=ZOOM,
    method_name=method_name
)
plt.savefig(f"{method_name.replace(' ', '_')}_{suffix}.png", dpi=400)

# Huggingface inpainting

https://huggingface.co/runwayml/stable-diffusion-inpainting

**Note**:
- Changes image scale => need to identify location of mask based on scaling...

In [ ]:
from diffusers import StableDiffusionInpaintPipeline
import torch

fig, ax = plt.subplots(1, 2)

vis = background_overlay.copy()
vis_ = vis.copy()
for i in range(vis_.shape[0]):
    for j in range(vis_.shape[1]):
        vis_[i, j] = vis[i, j] if mask[i, j] == 0 else 1

ax[0].imshow(vis)
ax[0].set_xlim(*ZOOM[0])
ax[0].set_ylim(*ZOOM[1])
ax[1].imshow(vis_)
ax[1].set_xlim(*ZOOM[0])
ax[1].set_ylim(*ZOOM[1])

mask_image = (mask_3d * 255).astype(np.uint8)
fig, ax = plt.subplots()
cs = plt.imshow(mask_image)
fig.colorbar(cs);

In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    # revision="fp16",
    # torch_dtype depends on CPU (float32) vs GPU (float16)
    # https://stackoverflow.com/questions/75641074/i-run-stable-diffusion-its-wrong-runtimeerror-layernormkernelimpl-not-implem
    # torch_dtype=torch.float16,
).to("cuda")
# Image and mask_image should be PIL images.
# The mask structure is white for in-painting and black for keeping as is
image = pipe(
    prompt="", 
    image=Image.fromarray(background_overlay),
    mask_image=Image.fromarray(mask_image)
).images[0]
image.save(os.path.join(DATA_DIR, "output_huggingface.jpg"))


In [ ]:
background_huggingface = cv.imread(os.path.join(DATA_DIR, "output_huggingface.jpg"), cv.IMREAD_COLOR)
background_huggingface = cv.cvtColor(background_huggingface, cv.COLOR_RGB2BGR)

In [ ]:
method_name = "Overlay HuggingFace Inpaint"
show_result(
    patch,
    background,
    background_huggingface,
    zoom=ZOOM,
    method_name=method_name
)
plt.savefig(f"{method_name.replace(' ', '_')}_{suffix}.png", dpi=400)

# OpenAI inpainting

In [ ]:
from openai import AzureOpenAI, OpenAI

In [ ]:
# client = AzureOpenAI(
#     api_version="2023-12-01-preview",
#     azure_endpoint="https://sdsc-hackathon-alpine-aster-13.openai.azure.com/",
#     api_key="977a2326c8c547b98666e6d88ed61c40",
#     azure_deployment="dall-e-3",
# )

In [ ]:
client = OpenAI(
    api_key="",
)

In [ ]:
# result = client.images.generate(
#     model="dall-e-3", # the name of your DALL-E 3 deployment
#     prompt="Clouds",
#     n=1
# )

In [ ]:
# For convenience, write images to disk and read them in the call
cv.imwrite(os.path.join(DATA_DIR, "input_dalle_image.png"), cv.cvtColor(background_overlay, cv.COLOR_RGB2BGR))
cv.imwrite(os.path.join(DATA_DIR, "input_dalle_image.jpg"), cv.cvtColor(background_overlay, cv.COLOR_RGB2BGR))
img_mask = cv.threshold(mask_3d, 0.9, 255, cv.THRESH_BINARY)[1]
cv.imwrite(os.path.join(DATA_DIR, "input_dalle_mask.png"), img_mask)

In [ ]:
response = client.images.edit(
    model="dall-e-2",
    image=open(os.path.join(DATA_DIR, "input_dalle_image.png"), "rb"),
    mask=open(os.path.join(DATA_DIR, "input_dalle_mask.png"), "rb"),
    prompt="Please perform content aware filling",
    n=1,
    size="512x512"
)
image_url = response.data[0].url

In [ ]:
img = cv.imread(os.path.join(DATA_DIR, "output_huggingface.jpg"), cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
plt.imshow(img)